## Using scMulan to annotate cell types in Heart, Lung, Liver, Bone marrow, Blood, Brain, and Thymus

#### we provide a liver dataset sampled (percentage of 20%) from Suo C, 2022 (doi/10.1126/science.abo0510)
you can download the sampled dataset for this tutorial from: https://cloud.tsinghua.edu.cn/f/45a7fd2a27e543539f59/?dl=1  
ckpt could be downloaded from: https://cloud.tsinghua.edu.cn/f/2250c5df51034b2e9a85/?dl=1

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3" ## set your available devices, each use ~2G GPU-MEMORY
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # if use CPU only
import scanpy as sc

import scMulan
from scMulan import GeneSymbolUniform

## 1. load h5ad
It's recommended that you use h5ad here with raw count (and after your QC)

In [2]:
adata = sc.read('Data/100_per_sample.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 60345 × 2000
    obs: 'biosample_id', 'donor_id', 'disease', 'sex', 'age', 'cell_type', 'sub_cluster', 'cellbender_ncount', 'cellbender_ngenes', 'cellranger_percent_mito', 'exon_prop', 'cellbender_entropy', 'cellranger_doublet_scores'
    uns: 'log1p'

## 2. transform original h5ad with uniformed genes (42117 genes)

This step transform the genes in input adata to 42117 gene symbols and reserves the corresponding gene expression values.

In [37]:
adata_GS_uniformed = GeneSymbolUniform(input_adata=adata,
                                 output_dir="Data/",
                                 output_prefix='simonson_ready_for_jupyter')

The shape of query data is: (60345, 36601)
The length of reference gene_list is: 42117
Performing gene symbol uniform, this step may take several minutes


Processing: 100%|██████████████████████████████████████| 36601/36601 [00:43<00:00, 845.44it/s]

Building output data, this step may take several minutes



Processing: 100%|█████████████████████████████████████| 24024/24024 [00:04<00:00, 5114.35it/s]


Shape of output data is (60345, 42117). It should have 42117 genes with cell number unchanged.
h5ad file saved in:/Users/danrongli/Desktop/use_scMulan/scMulan/Data/simonson_ready_for_jupyter_uniformed.h5ad
report file saved in: /Users/danrongli/Desktop/use_scMulan/scMulan/Data/simonson_ready_for_jupyter_report.csv


## 3. process uniformed data (simply norm and log1p)

In [5]:
## you can read the saved uniformed adata

#adata_GS_uniformed=sc.read_h5ad('Data/liver_uniformed.h5ad')

In [38]:
sc.read_h5ad('Data/simonson_ready_for_jupyter_uniformed.h5ad')

AnnData object with n_obs × n_vars = 60345 × 42117
    obs: 'biosample_id', 'donor_id', 'disease', 'sex', 'age', 'cell_type', 'sub_cluster', 'cellbender_ncount', 'cellbender_ngenes', 'cellranger_percent_mito', 'exon_prop', 'cellbender_entropy', 'cellranger_doublet_scores'

In [39]:
adata_GS_uniformed

AnnData object with n_obs × n_vars = 60345 × 42117
    obs: 'biosample_id', 'donor_id', 'disease', 'sex', 'age', 'cell_type', 'sub_cluster', 'cellbender_ncount', 'cellbender_ngenes', 'cellranger_percent_mito', 'exon_prop', 'cellbender_entropy', 'cellranger_doublet_scores'

In [40]:
# norm and log1p count matrix
if adata_GS_uniformed.X.max() > 10:
    print("here")
    sc.pp.normalize_total(adata_GS_uniformed, target_sum=1e4) 
    sc.pp.log1p(adata_GS_uniformed)

here


In [41]:
adata_GS_uniformed.X

<60345x42117 sparse matrix of type '<class 'numpy.float32'>'
	with 51383020 stored elements in Compressed Sparse Row format>

In [42]:
adata_GS_uniformed.var_names

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A3GALT2',
       ...
       'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11AP1', 'ZYG11B', 'ZYX', 'ZYXP1',
       'ZZEF1', 'ZZZ3'],
      dtype='object', length=42117)

In [43]:
adata_GS_uniformed.obs_names

Index(['ACCAACAAGGGTTTCT-1-1', 'ATTCACTTCCCGAGGT-1-1', 'ATGCATGCACGCTTAA-1-1',
       'CAACCTCTCAGACCCG-1-1', 'CTCCCAAGTTCGTAAC-1-1', 'TTCGCTGGTACTAGCT-1-1',
       'TACCTGCTCCTTCTTC-1-1', 'TAACGACAGTCTTGGT-1-1', 'TATTCCAGTCTCCTGT-1-1',
       'TCACTATTCTGGGCAC-1-1',
       ...
       'CATGGTATCTTTGATC-1-14', 'TCGACGGGTCAAATCC-1-14',
       'GGCTTGGGTGCAATGG-1-14', 'TCGGTCTCACTGATTG-1-14',
       'CCTTGTGTCGGAATTC-1-14', 'TCGTAGACACACTTAG-1-14',
       'TGAATGCTCATGCCAA-1-14', 'GTGGAGATCGGCTGAC-1-14',
       'TGAGTCAGTGGGCTCT-1-14', 'GTGGAGAAGAAATTGC-1-14'],
      dtype='object', length=60345)

In [44]:
adata_GS_uniformed.obs

,biosample_id,donor_id,disease,sex,age,cell_type,sub_cluster,cellbender_ncount,cellbender_ngenes,cellranger_percent_mito,exon_prop,cellbender_entropy,cellranger_doublet_scores
ACCAACAAGGGTTTCT-1-1,1452,P1452,NF,female,47,Cardiomyocyte cell,NA,9883.0,3484,0.000390,0.068495,7.625137,0.050334
ATTCACTTCCCGAGGT-1-1,1452,P1452,NF,female,47,Cardiomyocyte cell,NA,9817.0,3556,0.000587,0.115108,7.490208,0.075205
ATGCATGCACGCTTAA-1-1,1452,P1452,NF,female,47,Cardiomyocyte cell,NA,9877.0,3675,0.000294,0.113345,7.610279,0.096886
CAACCTCTCAGACCCG-1-1,1452,P1452,NF,female,47,Cardiomyocyte cell,NA,9824.0,3500,0.000295,0.111798,7.559907,0.101604
CTCCCAAGTTCGTAAC-1-1,1452,P1452,NF,female,47,Cardiomyocyte cell,NA,9759.0,3675,0.000495,0.115510,7.585167,0.157263
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGTAGACACACTTAG-1-14,1801,P1801,NF,male,42,Capillary endothelial cell,EC-General capillary,317.0,271,0.012719,0.096614,7.325364,0.013812
TGAATGCTCATGCCAA-1-14,1801,P1801,NF,male,42,Pericyte,NA,399.0,347,0.003268,0.113017,7.674779,0.016770
GTGGAGATCGGCTGAC-1-14,1801,P1801,NF,male,42,Lymphoid cell,NA,231.0,203,0.003273,0.116838,7.207568,0.076133
TGAGTCAGTGGGCTCT-1-14,1801,P1801,NF,male,42,Macrophage,NA,263.0,238,0.038321,0.184397,7.599493,0.193600


In [8]:
adata_GS_uniformed = sc.read_h5ad("Data/balanced_simonson_ready_for_jupyter_before_embedding.h5ad")

In [2]:
adata_GS_uniformed = sc.read_h5ad("Data/ready_for_scMulan_25.h5ad")

In [7]:
adata_GS_uniformed = sc.read_h5ad("Data/ready_for_scMulan_100.h5ad")

In [12]:
adata_GS_uniformed = sc.read_h5ad("Data/ready_for_scMulan_1000.h5ad")

In [2]:
adata_GS_uniformed = sc.read('Data/100_per_sample.h5ad')

In [2]:
adata_GS_uniformed = sc.read('Data/ready_for_input_scMulan.h5ad')

In [3]:
# you should first download ckpt from https://cloud.tsinghua.edu.cn/f/2250c5df51034b2e9a85/?dl=1
# put it under .ckpt/ckpt_scMulan.pt
# by: wget https://cloud.tsinghua.edu.cn/f/2250c5df51034b2e9a85/?dl=1  -O ckpt/ckpt_scMulan.pt

ckp_path = 'ckpt/ckpt_scMulan.pt'

In [4]:
scml = scMulan.model_inference(ckp_path, adata_GS_uniformed)
base_process = scml.cuda_count()

2024-10-22 21:55:00.635 | INFO     | scMulan.model.model:__init__:119 - number of parameters: 368.80M


✅ adata passed check
👸 scMulan is ready
scMulan is currently available to 0 GPUs.


In [5]:
#scml.get_cell_types_and_embds_for_adata(parallel=True, n_process = base_process)
scml.get_cell_types_and_embds_for_adata(parallel=False) # for only using CPU, but it is really slow.

⏳ Collecting cell embeddings for each cell: 100%|█| 60345/60345 [9:28:59<00:00,


In [6]:
scml.adata.write("results_scMulan.h5ad")

In [16]:
scml.adata.write("select_1000_scMulan.h5ad")

In [11]:
scml.adata.write("select_100_scMulan.h5ad")

In [6]:
scml.adata.write("select_25_scMulan.h5ad")

In [ ]:
scml.adata.write("100_per_sample_result.h5ad")

The predicted cell types are stored in scml.adata.obs['cell_type_from_scMulan'], besides the cell embeddings (for multibatch integration) in scml.adata.obsm['X_scMulan'] (not used in this tutorial).

In [12]:
scml.adata.write("balanced_after_select_genes.h5ad")

In [26]:
scml.adata.obs.to_csv("scMulan_obs.csv")

In [48]:
scml.adata.write("scMulan_another.h5ad")

In [20]:
scml.adata.obs['cell_type']

ACCAACAAGGGTTTCT-1-1     Cardiomyocyte cell
ATTCACTTCCCGAGGT-1-1     Cardiomyocyte cell
ATGCATGCACGCTTAA-1-1     Cardiomyocyte cell
CAACCTCTCAGACCCG-1-1     Cardiomyocyte cell
CTCCCAAGTTCGTAAC-1-1     Cardiomyocyte cell
                                ...        
TCGTAGACACACTTAG-1-14      Endothelial cell
TGAATGCTCATGCCAA-1-14              Pericyte
GTGGAGATCGGCTGAC-1-14         Lymphoid cell
TGAGTCAGTGGGCTCT-1-14            Macrophage
GTGGAGAAGAAATTGC-1-14      Endothelial cell
Name: cell_type, Length: 60345, dtype: category
Categories (11, object): ['Adipocyte', 'Basement membrane fibroblast', 'Cardiomyocyte cell', 'Endothelial cell', ..., 'Mesothelial cell', 'Neuron', 'Pericyte', 'Unclassified']

In [16]:
scml.adata.obs['cell_type_from_scMulan']

ACCAACAAGGGTTTCT-1-1             Cardiomyocyte cell
ATTCACTTCCCGAGGT-1-1             Cardiomyocyte cell
ATGCATGCACGCTTAA-1-1             Cardiomyocyte cell
CAACCTCTCAGACCCG-1-1             Cardiomyocyte cell
CTCCCAAGTTCGTAAC-1-1             Cardiomyocyte cell
                                    ...            
TCGTAGACACACTTAG-1-14    Capillary endothelial cell
TGAATGCTCATGCCAA-1-14                      Pericyte
GTGGAGATCGGCTGAC-1-14     Migratory dendriitic cell
TGAGTCAGTGGGCTCT-1-14                    Macrophage
GTGGAGAAGAAATTGC-1-14              Endothelial cell
Name: cell_type_from_scMulan, Length: 60345, dtype: object

In [17]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [21]:
y_true = scml.adata.obs['cell_type']
y_pred = scml.adata.obs['cell_type_from_scMulan']

In [22]:
# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate weighted precision
weighted_precision = precision_score(y_true, y_pred, average='weighted')

# Calculate weighted F1 score
weighted_f1 = f1_score(y_true, y_pred, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Weighted Precision: {weighted_precision}")
print(f"Weighted F1 Score: {weighted_f1}")

/Users/danrongli/miniforge3/envs/scMulan/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6983345761869252
Weighted Precision: 0.9728234248187101
Weighted F1 Score: 0.7179224131798324


## 5. visualization

In [ ]:
adata_mulan = scml.adata.copy()

In [ ]:
sc.pp.pca(adata_mulan)
sc.pl.pca_variance_ratio(adata_mulan)
sc.pp.neighbors(adata_mulan,n_pcs=10)
sc.tl.umap(adata_mulan)

In [ ]:
# you can run smoothing function to filter the false positives
scMulan.cell_type_smoothing(adata_mulan, threshold=0.1)

In [ ]:
# cell_type_from_scMulan: pred
# cell_type_from_mulan_smoothing: pred+smoothing
# original_name: original annotations by the authors
# cell_type: cell types in hECA-10M that maps original_name to uHAF

sc.pl.umap(adata_mulan,color=["cell_type_from_scMulan","cell_type_from_mulan_smoothing",'cell_type','original_name'],ncols=1)

In [ ]:
top_celltypes = adata_mulan.obs.cell_type_from_scMulan.value_counts().index[:20]

In [ ]:
# you can select some cell types of interest (from scMulan's prediction) for visulization
# selected_cell_types = ["NK cell", "Kupffer cell", "Conventional dendritic cell 2"] # as example
selected_cell_types = top_celltypes
scMulan.visualize_selected_cell_types(adata_mulan,selected_cell_types,smoothing=True)